In [256]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from operator import itemgetter

In [257]:
train_data = pd.read_csv('Updated_DataFrame_Train.csv')
test_data = pd.read_csv('Updated_DataFrame_Test.csv')

In [258]:
pd.set_option('display.max_columns', 52)
train_data.head()

,id,gender,area,qualification,income,marital_status,vintage,claim_amount,num_policies,policy,type_of_policy,cltv,income_int,Age,Education,Occupation,Income_velocity,num_policies_int,Num_claims,Claim_amount_per_policy,Claims_ratio,Loyalty_score,Ratio_claim_amount_to_income,Ratio_num_policies_to_vintage,Probability_claim_amount_given_income,Probability_num_policies_given_vintage,age,family_size,no_of_claims,spending_power,education_level,policy_tenure,avg_premium_amount,avg_claim_per_id,claim_amt_to_income_ratio,no_of_policies_to_vintage_ratio,premium_amt_to_income_ratio,no_of_claims_to_vintage_ratio,income_vintage,claim_num_policies,Ratio_vintage_to_policy,Frequency_gender,Frequency_area,Frequency_qualification,Frequency_income,Frequency_marital_status,Frequency_vintage,Frequency_claim_amount,Frequency_num_policies,Frequency_type_of_policy,Frequency_cltv,Probability_vintage_given_policy
0,1,Male,Urban,Bachelor,5L-10L,1,5,5790,More than 1,A,Platinum,64308,75000,2018,2,3,6250.0,2,1,2895.0,0.5,0.5,0.077200,0.400000,4174.886391,1.652832,0.013689,2,89392,0,2,3,2.542901,0.0,0.077200,0.400000,0.000040,17878.400000,375000,11580,1.666667,50497,62455,39399,52716,51444,12481,19,60263,47796,30,4.467216
1,2,Male,Rural,High School,5L-10L,0,8,5080,More than 1,A,Platinum,515400,75000,2015,1,3,6250.0,2,1,2540.0,0.5,0.8,0.067733,0.250000,4174.886391,1.755378,0.021903,1,89392,0,1,3,2.542901,5790.0,0.067733,0.250000,0.000040,11174.000000,600000,10160,2.666667,50497,26937,46247,52716,37948,7624,11,60263,47796,2,4.467216
2,3,Male,Urban,Bachelor,5L-10L,1,8,2599,More than 1,A,Platinum,64212,75000,2015,2,3,6250.0,2,1,1299.5,0.5,0.8,0.034653,0.250000,4174.886391,1.755378,0.021903,2,89392,0,2,3,2.542901,5080.0,0.034653,0.250000,0.000040,11174.000000,600000,5198,2.666667,50497,62455,39399,52716,51444,7624,18,60263,47796,36,4.467216
3,4,Female,Rural,High School,5L-10L,0,7,0,More than 1,A,Platinum,97920,75000,2016,1,3,6250.0,2,0,0.0,0.0,0.7,0.000000,0.285714,4174.886391,1.688736,0.019165,1,89392,0,1,3,2.542901,2599.0,0.000000,0.285714,0.000040,12770.285714,525000,0,2.333333,38895,26937,46247,52716,37948,13095,17671,60263,47796,13,4.467216
4,5,Male,Urban,High School,More than 10L,1,6,3508,More than 1,A,Gold,59736,120000,2017,1,4,10000.0,2,1,1754.0,0.5,0.6,0.029233,0.333333,3410.056133,1.667792,0.016427,2,89392,1000000,1,3,2.542901,0.0,0.029233,0.333333,0.000025,14898.666667,720000,7016,2.000000,50497,62455,46247,13664,51444,17769,16,60263,20586,19,4.467216


In [259]:
income = train_data['income']
area = train_data['area']
train_data['spending_power'] =  np.where((area=='Urban') & (income=='5L - 10L'), 150000,
                        np.where((area=='Urban') & (income=='2L - 5L'), 350000,
                        np.where((area=='Urban') & (income=='<=2L'), 200000, 
                        np.where((area=='Urban') & (income=='More than 10L'), 1000000,
                        np.where((area=='Rural') & (income=='5L - 10L'), 100000,
                        np.where((area=='Rural') & (income=='2L - 5L'), 350000,
                        np.where((area=='Rural') & (income=='<=2L'), 200000, 
                        np.where((area=='Rural') & (income=='More than 10L'), 1000000, 0))))))))

In [260]:
income = test_data['income']
area = test_data['area']
test_data['spending_power'] =  np.where((area=='Urban') & (income=='5L - 10L'), 150000,
                        np.where((area=='Urban') & (income=='2L - 5L'), 350000,
                        np.where((area=='Urban') & (income=='<=2L'), 200000, 
                        np.where((area=='Urban') & (income=='More than 10L'), 1000000,
                        np.where((area=='Rural') & (income=='5L - 10L'), 100000,
                        np.where((area=='Rural') & (income=='2L - 5L'), 350000,
                        np.where((area=='Rural') & (income=='<=2L'), 200000, 
                        np.where((area=='Rural') & (income=='More than 10L'), 1000000, 0))))))))

In [269]:
train_data.drop(['Frequency_gender', 'Frequency_area', 'Frequency_qualification', 'Frequency_income', 'Frequency_marital_status',
            'Frequency_vintage', 'Frequency_num_policies', 'Frequency_type_of_policy', 'Frequency_cltv', 'qualification', 'age',
            'num_policies_int', 'Occupation', 'Education', 'income_int', 'no_of_claims', 'family_size', 'spending_power'], 
            axis=1, inplace=True)

In [270]:
test_data.drop(['Frequency_gender', 'Frequency_area', 'Frequency_qualification', 'Frequency_income', 'Frequency_marital_status',
            'Frequency_vintage', 'Frequency_num_policies', 'Frequency_type_of_policy', 'qualification', 'age',
            'num_policies_int', 'Occupation', 'Education', 'income_int', 'no_of_claims', 'family_size', 'spending_power'], 
            axis=1, inplace=True)

In [266]:
cat_col = [col for col in train_data.columns if train_data[col].dtypes == 'object']
cat_col

[]

In [262]:
cat_col = [col for col in train_data.columns if train_data[col].dtypes == 'object']
for col in cat_col:
    le = LabelEncoder()
    train_data.loc[:, col] = le.fit_transform(train_data[col])

In [242]:
'''def frequency_encoding(df, cols):
    for col in cols:
        freq = df[col].value_counts()
        df[col] = df[col].map(freq)
    return df

# example usage
cols_to_encode = cat_col
train_data = frequency_encoding(train_data, cols_to_encode)'''

'def frequency_encoding(df, cols):\n    for col in cols:\n        freq = df[col].value_counts()\n        df[col] = df[col].map(freq)\n    return df\n\n# example usage\ncols_to_encode = cat_col\ntrain_data = frequency_encoding(train_data, cols_to_encode)'

In [243]:
'''def target_encoding(df,cols,target):
    for col in cols:
        means = df.groupby(col)[target].mean()
        df[col] = df[col].map(means)
    return df

# example usage
cols_to_encode = cat_col
target_col = 'cltv'
train_data = target_encoding(train_data, cols_to_encode,target_col)'''

"def target_encoding(df,cols,target):\n    for col in cols:\n        means = df.groupby(col)[target].mean()\n        df[col] = df[col].map(means)\n    return df\n\n# example usage\ncols_to_encode = cat_col\ntarget_col = 'cltv'\ntrain_data = target_encoding(train_data, cols_to_encode,target_col)"

In [244]:
'''train_data = pd.get_dummies(train_data, columns=cat_col)'''

'train_data = pd.get_dummies(train_data, columns=cat_col)'

In [264]:
train_data.head()

,id,gender,area,qualification,income,marital_status,vintage,claim_amount,num_policies,policy,type_of_policy,cltv,income_int,Age,Education,Occupation,Income_velocity,num_policies_int,Num_claims,Claim_amount_per_policy,Claims_ratio,Loyalty_score,Ratio_claim_amount_to_income,Ratio_num_policies_to_vintage,Probability_claim_amount_given_income,Probability_num_policies_given_vintage,age,family_size,no_of_claims,spending_power,education_level,policy_tenure,avg_premium_amount,avg_claim_per_id,claim_amt_to_income_ratio,no_of_policies_to_vintage_ratio,premium_amt_to_income_ratio,no_of_claims_to_vintage_ratio,income_vintage,claim_num_policies,Ratio_vintage_to_policy,Frequency_gender,Frequency_area,Frequency_qualification,Frequency_income,Frequency_marital_status,Frequency_vintage,Frequency_claim_amount,Frequency_num_policies,Frequency_type_of_policy,Frequency_cltv,Probability_vintage_given_policy
0,1,1,1,0,1,1,5,5790,1,0,1,64308,75000,2018,2,3,6250.0,2,1,2895.0,0.5,0.5,0.077200,0.400000,4174.886391,1.652832,0.013689,2,89392,0,2,3,2.542901,0.0,0.077200,0.400000,0.000040,17878.400000,375000,11580,1.666667,50497,62455,39399,52716,51444,12481,19,60263,47796,30,4.467216
1,2,1,0,1,1,0,8,5080,1,0,1,515400,75000,2015,1,3,6250.0,2,1,2540.0,0.5,0.8,0.067733,0.250000,4174.886391,1.755378,0.021903,1,89392,0,1,3,2.542901,5790.0,0.067733,0.250000,0.000040,11174.000000,600000,10160,2.666667,50497,26937,46247,52716,37948,7624,11,60263,47796,2,4.467216
2,3,1,1,0,1,1,8,2599,1,0,1,64212,75000,2015,2,3,6250.0,2,1,1299.5,0.5,0.8,0.034653,0.250000,4174.886391,1.755378,0.021903,2,89392,0,2,3,2.542901,5080.0,0.034653,0.250000,0.000040,11174.000000,600000,5198,2.666667,50497,62455,39399,52716,51444,7624,18,60263,47796,36,4.467216
3,4,0,0,1,1,0,7,0,1,0,1,97920,75000,2016,1,3,6250.0,2,0,0.0,0.0,0.7,0.000000,0.285714,4174.886391,1.688736,0.019165,1,89392,0,1,3,2.542901,2599.0,0.000000,0.285714,0.000040,12770.285714,525000,0,2.333333,38895,26937,46247,52716,37948,13095,17671,60263,47796,13,4.467216
4,5,1,1,1,3,1,6,3508,1,0,0,59736,120000,2017,1,4,10000.0,2,1,1754.0,0.5,0.6,0.029233,0.333333,3410.056133,1.667792,0.016427,2,89392,1000000,1,3,2.542901,0.0,0.029233,0.333333,0.000025,14898.666667,720000,7016,2.000000,50497,62455,46247,13664,51444,17769,16,60263,20586,19,4.467216


In [320]:
X = train_data.drop('cltv', axis=1)
y = train_data['cltv']

pd.set_option('display.max_rows', 67)
vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
pd.DataFrame({'vif': vif}, index=X.columns)

C:\Users\lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\regression\linear_model.py:1736: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


,vif
id,1.000300e+00
gender,1.019851e+00
area,1.415590e+00
income,inf
marital_status,1.029557e+00
vintage,inf
claim_amount,inf
num_policies,5.741437e+00
policy,inf
type_of_policy,1.017607e+00


In [321]:
X.drop(['Probability_vintage_given_policy', 'claim_num_policies', 'claim_amt_to_income_ratio', 'Ratio_claim_amount_to_income',
        'policy_tenure', 'Probability_claim_amount_given_income', 'policy_tenure', 'Loyalty_score', 'Claim_amount_per_policy'],
      axis=1, inplace=True)

In [322]:
X.drop(['Age'], axis=1, inplace=True)

In [323]:
X.drop(['vintage'], axis=1, inplace=True)

In [324]:
X.drop(['Num_claims'], axis=1, inplace=True)

In [325]:
X.drop(['Ratio_num_policies_to_vintage'], axis=1, inplace=True)

In [326]:
X.drop(['avg_premium_amount'], axis=1, inplace=True)

In [327]:
X.drop(['Probability_num_policies_given_vintage'], axis=1, inplace=True)

In [340]:
X.drop(['Ratio_vintage_to_policy'], axis=1, inplace=True)

In [342]:
X.drop(['income_vintage'], axis=1, inplace=True)

In [329]:
X.drop(['no_of_claims_to_vintage_ratio'], axis=1, inplace=True)

In [330]:
X.drop(['Claims_ratio'], axis=1, inplace=True)

In [344]:
vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
pd.DataFrame({'vif': vif}, index=X.columns)

,vif
id,3.618923
gender,2.255152
area,4.103869
income,2.621156
marital_status,2.266420
claim_amount,4.654316
num_policies,3.325195
policy,1.614807
type_of_policy,2.964587
education_level,5.784608


In [345]:
X.head()

,id,gender,area,income,marital_status,claim_amount,num_policies,policy,type_of_policy,education_level,avg_claim_per_id,no_of_policies_to_vintage_ratio,premium_amt_to_income_ratio,Frequency_claim_amount
0,1,1,1,1,1,5790,1,0,1,2,0.0,0.400000,0.000040,19
1,2,1,0,1,0,5080,1,0,1,1,5790.0,0.250000,0.000040,11
2,3,1,1,1,1,2599,1,0,1,2,5080.0,0.250000,0.000040,18
3,4,0,0,1,0,0,1,0,1,1,2599.0,0.285714,0.000040,17671
4,5,1,1,3,1,3508,1,0,0,1,0.0,0.333333,0.000025,16


In [339]:
X['Ratio_vintage_to_policy'].describe()

count    89392.000000
mean         2.026798
std          1.404259
min          0.000000
25%          1.000000
50%          2.000000
75%          2.666667
max          8.000000
Name: Ratio_vintage_to_policy, dtype: float64

In [346]:
X_constant = sm.add_constant(X)
lin_reg=sm.OLS(y, X_constant).fit()
lin_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   cltv   R-squared:                       0.150
Model:                            OLS   Adj. R-squared:                  0.150
Method:                 Least Squares   F-statistic:                     1123.
Date:                Sun, 22 Jan 2023   Prob (F-statistic):               0.00
Time:                        22:07:30   Log-Likelihood:            -1.1399e+06
No. Observations:               89392   AIC:                         2.280e+06
Df Residuals:                   89377   BIC:                         2.280e+06
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const                            3.688e+04   1544.485     23.877      0.000    3.39e+04    3.99e+04
id                                 -0.0111      0.011     -1.026      0.305      -0.032       0.010
gender                           2178.6427    568.097      3.835      0.000    1065.177    3292.108
area                             8820.5323    711.546     12.396      0.000    7425.908    1.02e+04
income                          -1885.5130    325.124     -5.799      0.000   -2522.752   -1248.274
marital_status                  -6695.4752    572.579    -11.694      0.000   -7817.725   -5573.225
claim_amount                        3.2580      0.119     27.468      0.000       3.026       3.491
num_policies                     6.479e+04    632.882    102.379      0.000    6.36e+04     6.6e+04
policy                            656.4250    453.153      1.449      0.147    -231.751    1544.601
type_of_policy                  -1716.1449    411.413     -4.171      0.000   -2522.510    -909.780
education_level                  1616.2324    492.976      3.279      0.001     650.005    2582.460
avg_claim_per_id                    0.0707      0.086      0.825      0.410      -0.097       0.239
no_of_policies_to_vintage_ratio   448.2172    664.607      0.674      0.500    -854.407    1750.841
premium_amt_to_income_ratio     -9.108e+06   1.12e+07     -0.816      0.415    -3.1e+07    1.28e+07
Frequency_claim_amount              0.3197      0.055      5.856      0.000       0.213       0.427
==============================================================================
Omnibus:                    51259.064   Durbin-Watson:                   1.995
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           404849.318
Skew:                           2.737   Prob(JB):                         0.00
Kurtosis:                      11.873   Cond. No.                     2.08e+09
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.08e+09. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
vif = [variance_inflation_factor(X_constant.values, i) for i in range(X_constant.shape[1])]
pd.DataFrame({'vif': vif}, index=X_constant.columns)

In [188]:
train_data.to_csv('new_train_frame.csv', index=False)
test_data.to_csv('new_test_frame.csv', index=False)

In [ ]:
train_data.corr()['cltv'].sort_values(ascending=False)

In [6]:
cat_cols = [col for col in train_data.columns if train_data[col].dtypes == 'object']
train_data = pd.get_dummies(data=train_data, columns=cat_cols)

cat_cols = [col for col in test_data.columns if test_data[col].dtypes == 'object']
test_data = pd.get_dummies(data=test_data, columns=cat_cols)

In [229]:
X = train_data.drop('cltv', axis=1)
y = train_data['cltv']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

regressor = LinearRegression()

In [230]:
n_features_to_select = 0.5
rfe = RFE(regressor, n_features_to_select=n_features_to_select)
rfe.fit(X_train, y_train)

RFE(estimator=LinearRegression(), n_features_to_select=0.5)

In [231]:
features = X_train.columns.to_list()
for x, y in (sorted(zip(rfe.ranking_ , features), key=itemgetter(0))):
    print(x, y)

n_features_to_select = 0.5
rfe = RFE(regressor, n_features_to_select=n_features_to_select)
rfe.fit(X_train, y_train)

predictions = rfe.predict(X_test)
print(predictions)

1 vintage
1 claim_amount
1 Age
1 Num_claims
1 Claim_amount_per_policy
1 Claims_ratio
1 Loyalty_score
1 Ratio_claim_amount_to_income
1 Ratio_num_policies_to_vintage
1 Probability_claim_amount_given_income
1 Probability_num_policies_given_vintage
1 claim_amt_to_income_ratio
1 no_of_policies_to_vintage_ratio
1 premium_amt_to_income_ratio
1 claim_num_policies
1 Probability_vintage_given_policy
2 income
3 Income_velocity
4 policy_tenure
5 policy
6 marital_status
7 education_level
8 Ratio_vintage_to_policy
9 Frequency_claim_amount
10 gender
11 num_policies
12 area
13 type_of_policy
14 no_of_claims_to_vintage_ratio
15 avg_claim_per_id
16 id
17 income_vintage
18 avg_premium_amount
[ 78763.8482666   65612.84008789  74923.50744629 ...  74280.70715332
  70449.45080566 119514.62268066]


In [15]:
boosting_regressor = GradientBoostingRegressor()

In [31]:
features = X_train.columns.to_list()
for x, y in (sorted(zip(rfe.ranking_ , features), key=itemgetter(0))):
    print(x, y)

n_features_to_select = 0.5
rfe = RFE(boosting_regressor, n_features_to_select=n_features_to_select)
rfe.fit(X_train, y_train)

predictions = rfe.predict(X_test)

output = pd.DataFrame({"Id": test_data.index, target:predictions})
output.to_csv('submission.csv', index=False)

1 marital_status
1 family_size
1 education_level
1 policy_tenure
1 gender_Male
1 area_Rural
1 area_Urban
1 qualification_Bachelor
1 qualification_High School
1 qualification_Others
1 num_policies_1
1 num_policies_More than 1
1 policy_A
1 policy_B
1 policy_C
1 type_of_policy_Gold
1 type_of_policy_Platinum
1 type_of_policy_Silver
2 gender_Female
3 premium_amt_to_income_ratio
4 claim_amt_to_income_ratio
5 income_2L-5L
6 income_Less than or equal to 2L
7 income_5L-10L
8 no_of_policies_to_vintage_ratio
9 vintage
10 claim_num_policies
11 claim_amount
12 age
13 no_of_claims_to_vintage_ratio
14 avg_claim_per_id
15 id
16 income_vintage
17 no_of_claims
18 spending_power
19 income_More than 10L
20 avg_premium_amount


KeyboardInterrupt: 

In [55]:
xgb_regressor = xgb.XGBRegressor()

In [56]:
n_features_to_select = 0.5
rfe = RFE(xgb_regressor, n_features_to_select=n_features_to_select)
rfe.fit(X_train, y_train)

RFE(estimator=XGBRegressor(base_score=None, booster=None, callbacks=None,
                           colsample_bylevel=None, colsample_bynode=None,
                           colsample_bytree=None, early_stopping_rounds=None,
                           enable_categorical=False, eval_metric=None,
                           gamma=None, gpu_id=None, grow_policy=None,
                           importance_type=None, interaction_constraints=None,
                           learning_rate=None, max_bin=None,
                           max_cat_to_onehot=None, max_delta_step=None,
                           max_depth=None, max_leaves=None,
                           min_child_weight=None, missing=nan,
                           monotone_constraints=None, n_estimators=100,
                           n_jobs=None, num_parallel_tree=None, predictor=None,
                           random_state=None, reg_alpha=None, reg_lambda=None, ...),
    n_features_to_select=0.5)

In [57]:
features = X_train.columns.to_list()
for x, y in (sorted(zip(rfe.ranking_ , features), key=itemgetter(0))):
    print(x, y)

n_features_to_select = 0.5
rfe = RFE(xgb_regressor, n_features_to_select=n_features_to_select)
rfe.fit(X_train, y_train)

predictions = rfe.predict(X_test)

output = pd.DataFrame({"Id": test_data.index, target:predictions})
output.to_csv('submission.csv', index=False)

1 id
1 marital_status
1 claim_amount
1 Education
1 Occupation
1 Income_velocity
1 Claim_amount_per_policy
1 Ratio_claim_amount_to_income
1 Ratio_num_policies_to_vintage
1 Probability_num_policies_given_vintage
1 spending_power
1 education_level
1 policy_tenure
1 avg_claim_per_id
1 claim_amt_to_income_ratio
1 premium_amt_to_income_ratio
1 no_of_claims_to_vintage_ratio
1 income_vintage
1 claim_num_policies
1 Ratio_vintage_to_policy
1 Frequency_gender
1 Frequency_area
1 Frequency_income
1 Frequency_vintage
1 Frequency_claim_amount
1 Frequency_num_policies
1 Frequency_type_of_policy
1 Frequency_cltv
1 Probability_vintage_given_policy
1 type_of_policy_Silver
2 vintage
3 Claims_ratio
4 policy_C
5 Num_claims
6 no_of_claims
7 Age
8 gender_Female
9 avg_premium_amount
10 Frequency_marital_status
11 gender_Male
12 policy_B
13 Frequency_qualification
14 type_of_policy_Gold
15 type_of_policy_Platinum
16 Loyalty_score
17 family_size
18 policy_A
19 no_of_policies_to_vintage_ratio
20 area_Rural
21 are

TypeError: unhashable type: 'Series'

In [11]:
X = df.drop('cltv', axis=1)

NameError: name 'df' is not defined

In [30]:
lasso = Lasso()

In [ ]:
n_features_to_select = 0.5
rfe = RFE(lasso, n_features_to_select=n_features_to_select)
rfe.fit(X_train, y_train)

In [ ]:
features = X_train.columns.to_list()
for x, y in (sorted(zip(rfe.ranking_ , features), key=itemgetter(0))):
    print(x, y)

n_features_to_select = 0.5
rfe = RFE(lasso, n_features_to_select=n_features_to_select)
rfe.fit(X_train, y_train)

predictions = rfe.predict(X_test)

output = pd.DataFrame({"Id": test_data.index, target:predictions})
output.to_csv('submission.csv', index=False)